In [3]:
# !pip install pdfminer.six
# !pip install pdfminer3
# !pip install pdfminer
# !pip install pyMuPDF
# !pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import pandas as pd
import matplotlib.pyplot as plt
import io
import unicodedata
import numpy as np
import re
import string

import sys, fitz
from pdf2image import convert_from_path
import easyocr
import numpy as np
import spacy
import pytesseract
import sklearn
import joblib

!pip install spacy
import spacy
spacy.prefer_gpu()
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
from spacy.matcher import Matcher
!pip install -U spacy

from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer
import re
from pdfminer.high_level import extract_text
import nltk
import warnings
warnings.filterwarnings('ignore')
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
!pip install spacy
import spacy
spacy.prefer_gpu()
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
from spacy.matcher import Matcher
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager,PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator,TextConverter

C:\Users\msmka\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [33]:
class GALKResumeParser(Model):

    def __init__(self):
        pass
   
    def get_name(self):
        nlp=spacy.load('output3/model-best')
        doc=nlp(self.text)
#         print(doc.ents)
        try:
            self.name=doc.ents[0]
        except:
            self.name="Name not found"
    def convertPDFtoTEXT(self,resume):
        """
        Function to extract text using pdfminer3 from PDF file.
        input:
            resume_file: string
            Tells the absolute path to the resume location.
        output:
            text: string
            Raw text that has been extracted from the PDF file.
        """
        resource_manager = PDFResourceManager()
        file_handle = io.StringIO()
        converter = TextConverter(resource_manager, file_handle, laparams=LAParams())
        page_interpreter = PDFPageInterpreter(resource_manager, converter)

        with open(resume, 'rb') as fh:
            for page in PDFPage.get_pages(fh,
                                          caching=True,
                                          check_extractable=True):
                page_interpreter.process_page(page)
            text = file_handle.getvalue()
        converter.close()
        file_handle.close()
        self.text=text

    def ProcessConverted(self,text):
        """
        Function that cleans the converted text and allows limited character types in it.
        Input:
            text: string
            Raw text to be processed
        Output:
            sentences: list
            List of text sentences after being processed 
        """
        sentences=[]
        allowed=["/","-",":","(",")",","," ","."]
        allowed.extend(list(string.ascii_letters+string.digits))
        for s in text.split("\n"):
            if(s==len(s)*" "):
                continue
            t=""
            for letter in s:
                if letter in allowed:
                    t+=letter
            t=t.strip()
            if(t):
                sentences.append(t)
        self.text="\n".join(sentences)
    
    def fit(self,resume_location):
        self.resume=resume_location
        self.convertPDFtoTEXT(self.resume)
        self.ProcessConverted(self.text)
        self.get_designation()
        self.Organizations()
        self.get_name()
#         self.name=""
        self.get_skills()
        self.spacy_ner()
        self.classifier_model()
    
    def common_func(self,RESERVED_WORDS,lst):
        organizations = []
        for sent in nltk.sent_tokenize(self.text):
            for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
                if hasattr(chunk, 'label'):
                    organizations.append(' '.join(c[0] for c in chunk.leaves()))
                elif chunk[1]=='IN':
                    organizations.append(chunk[0])
        a=[]
        a.append(organizations[0])
        for i in range(1,len(organizations)-1):
            if organizations[i] in lst:
                a.append(organizations[i-1]+' '+organizations[i]+' '+organizations[i+1])
            else:
                a.append(organizations[i])
        a.append(organizations[len(organizations)-1])
        
        education = set()
        for org in a:
            for i in org.split():
                if i.lower() in RESERVED_WORDS:
                    education.add(org)
        education=list(education)
        final=set()
        for i in education:
            final.add(i)
        return final
    def Organizations(self):
        IITS = ['IIT Madras','IIT Delhi','IIT Bombay','IIT Kanpur','IIT Kharagpur','IIT Roorkee','IIT Guwahati','IIT Hyderabad','IIT Dhanbad','IIT Indore','IIT Varanasi','IIT BHU','IIT Ropar','IIT Patna','IIT Gandhinagar','IIT Bhubaneswar','IIT Mandi','IIT Jodhpur','IIT Tirupati','IIT Bhilai','IIT Goa','IIT Jammu','IIT Dharwad','IIT Palakkad']
        Indian_Institute = ['Indian Institute of Technology '+s[4:] for s in IITS]
        Institution = IITS+Indian_Institute
        EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
             'B. Tech', 'M. Tech','SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
        ]
        RESERVED_WORDS = [
            "class",
            "institute of",
            "b.tech",
            "year",
            "national institute",
            "indian institute of technology",
            "b.sc",
            "IIT",
            "technology",
            'school',
            'college',
            'university',
            "education",
            'academy',
            'faculty',
            'institute',
            'faculdades',
            'Schola',
            'schule',
            'lise',
            'lyceum',
            'lycee',
            'polytechnic',
            'kolej',
            'ünivers',
            'okul',
            *EDUCATION,
            *Institution
        ]
        
        self.org=self.common_func(RESERVED_WORDS,["of"])
    
    def get_designation(self):
        RESERVED_WORDS = [
            'assistant',
            'associate',
            'analyst',
            'apprentice',
            'intern',
            'internship',
            'manager',
            'founder',
            'cofounder',
            'core',
            'coordinator',
            'volunteer',
            'head',
            'president',
            'consultant',
            'developer',
            'lead'
        ]
        self.designation=self.common_func(RESERVED_WORDS,['of','at','in','for','to'])
    def spacy_ner(self):
        Skills = ['Flask','Django','Mysql','C','SQL','Css','html','js','Machine learning','C++','Algorithms','Github','Php','Python','opencv','deep learning','autocad','simulink','tensorflow','matlab','keras','numpy','nlp','data science','optimization','operational research',
          'Natural Language Processing','probability','statistics','data analytics','Data Engineering','Data structures','sklearn','pandas','nltk','spacy']
        Projects = ['Logistic Regression','Recommendation System','Pipeline','Decision Tree','Random Forest','Forecast','neural networks','pipeline','segmentation','data mining','neural network',
             'Data acquisition','DATA ANALYSIS','FRAUD DETECTION','Knn classifier','Support Vector Machine']
        Experience = ['Team Lead','Intern','Volunteer','ENGINEER','MACHINE LEARNING INTERN','DATA SCIENCE INTERN','SDE INTERN','BUSINESS DEVELOPMENT INTERN','FINANCE INTERN',"Leader",'Data Scientist']

        IITS = ['IIT Madras','IIT Delhi','IIT Bombay','IIT Kanpur','IIT Kharagpur','IIT Roorkee','IIT Guwahati','IIT Hyderabad','IIT Dhanbad','IIT Indore','IIT Varanasi','IIT BHU','IIT Ropar','IIT Patna','IIT Gandhinagar','IIT Bhubaneswar','IIT Mandi','IIT Jodhpur','IIT Tirupati','IIT Bhilai','IIT Goa','IIT Jammu','IIT Dharwad','IIT Palakkad']
        Indian_Institute = ['Indian Institute of Technology '+s[4:] for s in IITS]
        Institution = IITS+Indian_Institute
        EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
             'B. Tech', 'M. Tech','SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
        ]
        techskills = open("TechSkills.txt", "r").read().lower()
        techskills = set(techskills.split())
        Skills+=list(techskills)

        nontechskills = open("Nontech.txt", "r").read().lower()
        nontechskills = set(nontechskills.split())
        soft_skills=list(nontechskills)

        from spacy.lang.en import English


        config = {
           "phrase_matcher_attr":None,
           "validate": True,
          "overwrite_ents": False
    
        }
        nlp = spacy.load("en_core_web_sm")

        # move the NER component to the end of the pipeline: remove and then reload from the same source in the new position
        nlp.remove_pipe("ner")
        # nlp.add_pipe("ner", source=spacy.load("en_core_web_sm"))

        # add entity ruler
        ruler = nlp.add_pipe("entity_ruler")#, before="ner")

        #ruler = nlp.add_pipe("entity_ruler",config=config)
        patterns = [{"label": "MOBILE", "pattern": [{"TEXT": {"REGEX": r"\d{10}"}}]},
           {"label": "EMAIL", "pattern": [{"TEXT": {"REGEX": r'[A-Za-z0-9._%+-]+\s?@\s?[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}'}}]},
           {"label":"YEAR","pattern":[{"TEXT":{"REGEX":r"^(18|19|20)\d{2}$"}}]},
            {"label":'CGPA',"pattern":[{"TEXT":{"REGEX": r'\d[.]\d{2}'}}]}
           ]
        
        indianNames = open("allNames.txt", "r").read().lower()
        indianNames = set(indianNames.split())

        for skill in Skills:
            patterns.append({"label":"SKILLS","pattern":[{"LOWER":x.lower()} for x in skill.split()]})
        for skill in soft_skills:
            patterns.append({"label":"SOFT SKILLS","pattern":[{"LOWER":x.lower()} for x in skill.split()]})
    
        for project in Projects:
            patterns.append({"label":"PROJECT","pattern":[{"LOWER":x.lower()} for x in project.split()]})
    
        for institute in Institution:
            patterns.append({"label":"INSTITUTE","pattern":[{"LOWER":x.lower()} for x in institute.split()]})

        for ed in EDUCATION:
            patterns.append({"label":"EDUCATION","pattern":[{"LOWER":x.lower()} for x in ed.split()]})
    
        for exp in Experience:
            patterns.append({"label":"EXPERIENCE","pattern":[{"LOWER":exp.lower()}]})
    
        for name in indianNames:
            patterns.append({"label":"PERSON","pattern":[{"LOWER":name.lower()}]})

        ruler.add_patterns(patterns)

        doc = nlp(self.text)
        doc.ents = list(doc.ents)
        self.doc_ents = doc.ents
        
    def get_skills(self):
        db=pd.read_csv("Skills.csv")
        db.replace(np.nan,"no",inplace=True)
        SKILLS_DB =[]
        for el in db.values:
            SKILLS_DB.extend(el)
        for i in range(len(SKILLS_DB)):
            SKILLS_DB[i]=SKILLS_DB[i].lower()
        ConvertedText=self.text
        stop_words = set(nltk.corpus.stopwords.words('english'))
        word_tokens = nltk.tokenize.word_tokenize(ConvertedText)
        filtered_tokens = [w for w in word_tokens if w not in stop_words]
        filtered_tokens = [w for w in word_tokens if w.isalpha()]
        bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
        found_skills = set()
        for token in filtered_tokens:
            cols=db.columns
            exists=0
            for col in range(len(cols)):
                n=len(db.values[col])
                found=0
                matrix=cv.fit_transform([token.lower(),", ".join(db.values[col]).lower()])
                if cosine_similarity(matrix)[1][0]*100 > 15:
                    for i in range(n):
                        try:
                            matrix=cv.fit_transform([token.lower(),db.values[col][i].lower()])
                            similarity_matrix=cosine_similarity(matrix)
                            if(similarity_matrix[1][0]*100>80):
                                found_skills.add(token)
                                found=1
                                break
                        except:
                            pass
                else:
                    pass
        for ngram in bigrams_trigrams:
            if ngram.lower() in SKILLS_DB:
                found_skills.add(ngram)
        skills=set()
        for skill in found_skills:
            if skill not in list(stop_words):
                skills.add(skill.lower())
        self.Skills=skills
        return skills
    
    def classifier_model(self):
        
        df = pd.read_csv('UpdatedResumeDataSet.csv')
        def cleanResume(resumeText):
            resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
            resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
            resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
            resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
            resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
            resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
            resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
            return resumeText

        df['cleaned'] = df['Resume'].apply(lambda x:cleanResume(x))
        label = LabelEncoder()
        df['Category'] = label.fit_transform(df['Category'])

        le_name_mapping = dict(zip(label.classes_, label.transform(label.classes_)))
        le_name_mapping = {v:k for (k,v) in le_name_mapping.items()}

        df_text = df['cleaned'].values
        target = df['Category'].values
        word_vectorizer = TfidfVectorizer(
            sublinear_tf=True,
            stop_words='english',
            max_features=1500)
        word_vectorizer.fit(df_text)
        WordFeatures = word_vectorizer.transform(df_text)

        model = joblib.load('category_classifier.obj')
    
        WordFeatures = word_vectorizer.transform([self.text])
        y_pred = model.predict(WordFeatures.reshape(1,-1)) 

        self.Category = le_name_mapping[y_pred[0]]
    def predict(self):
        entities={}
        for ent in self.doc_ents:
            if ent.label_=='PERSON':
                continue
            entities.setdefault(ent.label_, set())
            entities[ent.label_].add(ent.text)
            
        Candidate = [ent.text for ent in self.doc_ents if ent.label_=='PERSON'][0]
        try:
            if self.name=="Name not found":
                entities['CANDIDATE'] = {Candidate}
        except:
            entities['CANDIDATE'] = {self.name}
        for skill in self.Skills:
            entities["SKILLS"].add(skill)
        for designation in self.designation:
            entities['EXPERIENCE'].add(designation)
        entities["ORGANIZATIONS"]=set()
        for orga in self.org:
            entities['ORGANIZATIONS'].add(orga)
        entities['JOB CATEGORY'] = self.Category    
        
        print("ENTITIES :")
        pp = pprint.PrettyPrinter(depth=4)
        pp.pprint(entities)
        
        return entities
        
            